In [1]:
import pandas as pd
import requests
import kaggle

How do I download using requests or API?
https://github.com/Kaggle/kaggle-api
kaggle datasets download -d christophercorrea/dc-residential-properties
print(perc_nan)

In [2]:
#how do I not write the entire file path?
data = pd.read_csv('/Users/Julia/dc-residential-properties/DC_Properties.csv', low_memory=False)
print(data.shape)
print(data.columns.tolist())
data.head(10)

(158957, 49)
['Unnamed: 0', 'BATHRM', 'HF_BATHRM', 'HEAT', 'AC', 'NUM_UNITS', 'ROOMS', 'BEDRM', 'AYB', 'YR_RMDL', 'EYB', 'STORIES', 'SALEDATE', 'PRICE', 'QUALIFIED', 'SALE_NUM', 'GBA', 'BLDG_NUM', 'STYLE', 'STRUCT', 'GRADE', 'CNDTN', 'EXTWALL', 'ROOF', 'INTWALL', 'KITCHENS', 'FIREPLACES', 'USECODE', 'LANDAREA', 'GIS_LAST_MOD_DTTM', 'SOURCE', 'CMPLX_NUM', 'LIVING_GBA', 'FULLADDRESS', 'CITY', 'STATE', 'ZIPCODE', 'NATIONALGRID', 'LATITUDE', 'LONGITUDE', 'ASSESSMENT_NBHD', 'ASSESSMENT_SUBNBHD', 'CENSUS_TRACT', 'CENSUS_BLOCK', 'WARD', 'SQUARE', 'X', 'Y', 'QUADRANT']


,Unnamed: 0,BATHRM,HF_BATHRM,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,...,LONGITUDE,ASSESSMENT_NBHD,ASSESSMENT_SUBNBHD,CENSUS_TRACT,CENSUS_BLOCK,WARD,SQUARE,X,Y,QUADRANT
0,0,4,0,Warm Cool,Y,2.0,8,4,1910.0,1988.0,...,-77.040832,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW
1,1,3,1,Warm Cool,Y,2.0,11,5,1898.0,2007.0,...,-77.040764,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW
2,2,3,1,Hot Water Rad,Y,2.0,9,5,1910.0,2009.0,...,-77.040678,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW
3,3,3,1,Hot Water Rad,Y,2.0,8,5,1900.0,2003.0,...,-77.040629,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW
4,4,2,1,Warm Cool,Y,1.0,11,3,1913.0,2012.0,...,-77.039361,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW
5,5,3,2,Hot Water Rad,Y,1.0,10,5,1913.0,NaN,...,-77.039715,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW
6,6,1,0,Warm Cool,Y,2.0,5,2,1917.0,1988.0,...,-77.041055,Old City 2,040 D Old City 2,4201.0,004201 2005,Ward 2,0152,-77.040429,38.914881,NW
7,7,3,1,Hot Water Rad,Y,2.0,8,4,1906.0,2011.0,...,-77.040129,Old City 2,040 D Old City 2,4201.0,004201 2005,Ward 2,0152,-77.040429,38.914881,NW
8,8,3,1,Warm Cool,Y,2.0,7,3,1908.0,2008.0,...,-77.039903,Old City 2,040 D Old City 2,4201.0,004201 2005,Ward 2,0152,-77.040429,38.914881,NW
9,9,1,1,Hot Water Rad,Y,1.0,6,2,1908.0,1979.0,...,-77.040081,Old City 2,040 D Old City 2,4201.0,004201 2005,Ward 2,0152,-77.040429,38.914881,NW


#Removed duplicate column, 'Unnamed: 0'

In [3]:
ddat = pd.DataFrame.drop(data, columns = 'Unnamed: 0')

In [4]:
#combine BR columns
ddat['BATHRMS'] = ddat['BATHRM'] + ddat['HF_BATHRM']
ddat = pd.DataFrame.drop(ddat, columns = ['BATHRM', 'HF_BATHRM'])

In [5]:
#percentages of NaN values per column
perc_nan=(ddat.isnull().sum(axis = 0))/158957

In [6]:
#delete NaN rows from columns with few NaN values:
dlist = (perc_nan[perc_nan <= 0.02].index).tolist()
print(dlist)
ddat = ddat.dropna(subset=dlist)

['HEAT', 'AC', 'ROOMS', 'BEDRM', 'AYB', 'EYB', 'QUALIFIED', 'SALE_NUM', 'BLDG_NUM', 'FIREPLACES', 'USECODE', 'LANDAREA', 'GIS_LAST_MOD_DTTM', 'SOURCE', 'ZIPCODE', 'LATITUDE', 'LONGITUDE', 'ASSESSMENT_NBHD', 'CENSUS_TRACT', 'WARD', 'SQUARE', 'X', 'Y', 'QUADRANT', 'BATHRMS']


In [7]:
#evaluate rows with more NaN values
print(perc_nan[perc_nan > 0.02])

NUM_UNITS             0.328774
YR_RMDL               0.490881
STORIES               0.329051
SALEDATE              0.168410
PRICE                 0.382122
GBA                   0.328774
STYLE                 0.328774
STRUCT                0.328774
GRADE                 0.328774
CNDTN                 0.328774
EXTWALL               0.328774
ROOF                  0.328774
INTWALL               0.328774
KITCHENS              0.328781
CMPLX_NUM             0.671226
LIVING_GBA            0.671226
FULLADDRESS           0.332901
CITY                  0.332832
STATE                 0.332832
NATIONALGRID          0.332832
ASSESSMENT_SUBNBHD    0.204779
CENSUS_BLOCK          0.332832
dtype: float64


In [8]:
#Are values the same for CITY and STATE?
print(data.CITY.value_counts())
print(data.STATE.value_counts())

WASHINGTON    106051
Name: CITY, dtype: int64
DC    106051
Name: STATE, dtype: int64


In [9]:
ddat = pd.DataFrame.drop(ddat, columns=['CMPLX_NUM', 'STATE', 'CITY', 'FULLADDRESS', 'NATIONALGRID'])
print(ddat.shape)
print(ddat.columns.tolist())
ddat.head(10)


(158449, 42)
['HEAT', 'AC', 'NUM_UNITS', 'ROOMS', 'BEDRM', 'AYB', 'YR_RMDL', 'EYB', 'STORIES', 'SALEDATE', 'PRICE', 'QUALIFIED', 'SALE_NUM', 'GBA', 'BLDG_NUM', 'STYLE', 'STRUCT', 'GRADE', 'CNDTN', 'EXTWALL', 'ROOF', 'INTWALL', 'KITCHENS', 'FIREPLACES', 'USECODE', 'LANDAREA', 'GIS_LAST_MOD_DTTM', 'SOURCE', 'LIVING_GBA', 'ZIPCODE', 'LATITUDE', 'LONGITUDE', 'ASSESSMENT_NBHD', 'ASSESSMENT_SUBNBHD', 'CENSUS_TRACT', 'CENSUS_BLOCK', 'WARD', 'SQUARE', 'X', 'Y', 'QUADRANT', 'BATHRMS']


,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,EYB,STORIES,SALEDATE,...,ASSESSMENT_NBHD,ASSESSMENT_SUBNBHD,CENSUS_TRACT,CENSUS_BLOCK,WARD,SQUARE,X,Y,QUADRANT,BATHRMS
0,Warm Cool,Y,2.0,8,4,1910.0,1988.0,1972,3.0,2003-11-25 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW,4
1,Warm Cool,Y,2.0,11,5,1898.0,2007.0,1972,3.0,2000-08-17 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW,4
2,Hot Water Rad,Y,2.0,9,5,1910.0,2009.0,1984,3.0,2016-06-21 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW,4
3,Hot Water Rad,Y,2.0,8,5,1900.0,2003.0,1984,3.0,2006-07-12 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW,4
4,Warm Cool,Y,1.0,11,3,1913.0,2012.0,1985,3.0,NaN,...,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW,3
5,Hot Water Rad,Y,1.0,10,5,1913.0,NaN,1972,4.0,2010-02-26 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW,5
6,Warm Cool,Y,2.0,5,2,1917.0,1988.0,1957,2.0,2011-05-02 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2005,Ward 2,0152,-77.040429,38.914881,NW,1
7,Hot Water Rad,Y,2.0,8,4,1906.0,2011.0,1972,3.0,2011-09-29 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2005,Ward 2,0152,-77.040429,38.914881,NW,4
8,Warm Cool,Y,2.0,7,3,1908.0,2008.0,1967,2.0,2018-05-03 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2005,Ward 2,0152,-77.040429,38.914881,NW,4
9,Hot Water Rad,Y,1.0,6,2,1908.0,1979.0,1950,2.0,2008-12-05 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2005,Ward 2,0152,-77.040429,38.914881,NW,2


In [10]:
#We will make a small dataset, which contains no null values. 
sdat = ddat.dropna(subset=['PRICE'])
print(sdat.shape)
sdat.head(10)

(98003, 42)


,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,EYB,STORIES,SALEDATE,...,ASSESSMENT_NBHD,ASSESSMENT_SUBNBHD,CENSUS_TRACT,CENSUS_BLOCK,WARD,SQUARE,X,Y,QUADRANT,BATHRMS
0,Warm Cool,Y,2.0,8,4,1910.0,1988.0,1972,3.0,2003-11-25 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW,4
2,Hot Water Rad,Y,2.0,9,5,1910.0,2009.0,1984,3.0,2016-06-21 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW,4
3,Hot Water Rad,Y,2.0,8,5,1900.0,2003.0,1984,3.0,2006-07-12 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW,4
5,Hot Water Rad,Y,1.0,10,5,1913.0,NaN,1972,4.0,2010-02-26 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,0152,-77.040429,38.914881,NW,5
7,Hot Water Rad,Y,2.0,8,4,1906.0,2011.0,1972,3.0,2011-09-29 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2005,Ward 2,0152,-77.040429,38.914881,NW,4
8,Warm Cool,Y,2.0,7,3,1908.0,2008.0,1967,2.0,2018-05-03 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2005,Ward 2,0152,-77.040429,38.914881,NW,4
14,Warm Cool,Y,2.0,5,3,1917.0,2000.0,1967,2.0,2011-09-30 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2005,Ward 2,0152,-77.040429,38.914881,NW,4
16,Warm Cool,Y,1.0,8,3,1908.0,2006.0,1967,2.0,2010-11-04 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2005,Ward 2,0152,-77.040429,38.914881,NW,4
19,Hot Water Rad,Y,2.0,9,3,1908.0,2013.0,1969,2.0,2001-07-23 00:00:00,...,Old City 2,040 D Old City 2,4201.0,004201 2005,Ward 2,0152,-77.040429,38.914881,NW,4
20,Hot Water Rad,Y,1.0,14,5,1880.0,2009.0,1987,3.0,2016-01-12 00:00:00,...,Old City 2,040 E Old City 2,5201.0,005201 4003,Ward 2,0208,-77.033366,38.911716,NW,4


In [11]:
#if we remove all NaNs, we have no data.
ldat = sdat.dropna()
ldat.head()
#fill na vals using machine learning? 
#filling with average/mean/median doesn't make sense...


,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,EYB,STORIES,SALEDATE,...,ASSESSMENT_NBHD,ASSESSMENT_SUBNBHD,CENSUS_TRACT,CENSUS_BLOCK,WARD,SQUARE,X,Y,QUADRANT,BATHRMS


In [19]:
count = (sdat.isnull().sum(axis = 0))
nulcols = count[count != 0]
nullist = (nulcols.index).tolist()
nuldat = sdat[nullist]
print(nulcols)

NUM_UNITS             40299
YR_RMDL               40432
STORIES               40332
SALEDATE                  1
GBA                   40299
STYLE                 40299
STRUCT                40299
GRADE                 40299
CNDTN                 40299
EXTWALL               40299
ROOF                  40299
INTWALL               40299
KITCHENS              40300
LIVING_GBA            57704
ASSESSMENT_SUBNBHD    20207
CENSUS_BLOCK          40625
dtype: int64


In [ ]:
#drop one null from SALEDATE
sdat = sdat.dropna(subset=['SALEDATE'])

In [21]:
nulcols=nulcols.index.tolist()
nuldat = sdat[nulcols]

#Dataframe: nuldat
#How do I make a dataframe from nuldat which only includes string cols?
#nulstr
#How do I iterate value_counts for each col over dataframe?

        

In [ ]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [ ]:
#Dataframe: nuldat
#How do I make a dataframe from nuldat which only includes numerical cols?
#nulnum
#How do I iterate remove_outlier for each col over dataframe?
